In [1]:
# # # This Python 3 environment comes with many helpful analytics libraries installed
# # # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # # For example, here's several helpful packages to load

# # import numpy as np # linear algebra
# # import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # # Input data files are available in the read-only "../input/" directory
# # # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input/cache-meld'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# --- CELL 1: CÀI ĐẶT THƯ VIỆN (BẢN ỔN ĐỊNH) ---

# 1. Gỡ cài đặt phiên bản lỗi hiện tại
!pip uninstall -y transformers peft

# 2. Cài đặt phiên bản tương thích (Transformers 4.44.2 và PEFT mới nhất)
!pip install -q transformers==4.44.2
!pip install -q peft==0.12.0
!pip install -q huggingface_hub librosa

import os
os.environ['HF_HUB_DISABLE_TELEMETRY'] = '1'
print("✅ Đã cài đặt Transformers 4.44.2 & PEFT 0.12.0 (Tương thích hoàn toàn).")

Found existing installation: transformers 4.53.3
Uninstalling transformers-4.53.3:
  Successfully uninstalled transformers-4.53.3
Found existing installation: peft 0.16.0
Uninstalling peft-0.16.0:
  Successfully uninstalled peft-0.16.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 70.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 94.8 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 7.2 MB/s 

In [3]:
# --- CELL 2: IMPORT & CẤU HÌNH ---
import torch, numpy as np, pandas as pd
import torchaudio
from pathlib import Path
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, Wav2Vec2FeatureExtractor, Wav2Vec2Model
from sklearn.metrics import f1_score, accuracy_score
import torch.nn as nn
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

# Import LoRA
from peft import get_peft_model, LoraConfig, TaskType

# Config
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🟢 Device: {device}")

DATA_DIR = Path("/kaggle/input/meld-emotion-recognition/MELD.Raw/MELD.Raw")
TRAIN_CSV = DATA_DIR / "train/train_sent_emo.csv"
DEV_CSV = DATA_DIR / "dev_sent_emo.csv"
AUDIO_TRAIN = DATA_DIR / "train/train_splits"
AUDIO_DEV = DATA_DIR / "dev/dev_splits_complete"

TEXT_MODEL = "roberta-large"
AUDIO_MODEL = "ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition"

EMOTIONS = ['anger','disgust','fear','joy','neutral','sadness','surprise']
LABEL2ID = {e:i for i,e in enumerate(EMOTIONS)}

🟢 Device: cuda


In [4]:
# --- CELL 3: TRÍCH XUẤT AUDIO (GIỮ NGUYÊN VÌ NÓ ĐANG TỐT) ---
# Chúng ta vẫn đóng băng Audio để tiết kiệm VRAM cho LoRA Text
print("🚀 Đang tải Audio Model để trích xuất...")

feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(AUDIO_MODEL)
audio_model = Wav2Vec2Model.from_pretrained(AUDIO_MODEL).to(device)
audio_model.eval()

def extract_audio_features(csv_path, audio_dir):
    df = pd.read_csv(csv_path)
    features_dict = {}
    
    print(f"Processing audio for {csv_path.name}...")
    for _, row in tqdm(df.iterrows(), total=len(df)):
        uid = f"dia{row['Dialogue_ID']}_utt{row['Utterance_ID']}"
        file_path = audio_dir / f"{uid}.mp4"
        
        default_vec = np.zeros(1024, dtype=np.float32)
        
        if not file_path.exists():
            features_dict[uid] = default_vec
            continue
            
        try:
            speech, sr = torchaudio.load(file_path)
            if sr != 16000:
                resampler = torchaudio.transforms.Resample(sr, 16000)
                speech = resampler(speech)
            
            speech = speech.squeeze()
            if speech.ndim > 1: speech = speech.mean(dim=0)
            if len(speech) < 160:
                 features_dict[uid] = default_vec
                 continue

            inputs = feature_extractor(
                speech.numpy(), 
                sampling_rate=16000, 
                return_tensors="pt",
                padding="max_length",
                max_length=16000*4, 
                truncation=True
            )
            input_values = inputs.input_values.to(device)
            
            with torch.no_grad():
                outputs = audio_model(input_values)
                pooled_output = torch.mean(outputs.last_hidden_state, dim=1)
            
            vec = pooled_output.cpu().numpy().flatten()
            if vec.shape != (1024,): features_dict[uid] = default_vec
            else: features_dict[uid] = vec
            
        except Exception as e:
            features_dict[uid] = default_vec
            
    return features_dict

audio_features_train = extract_audio_features(TRAIN_CSV, AUDIO_TRAIN)
audio_features_dev = extract_audio_features(DEV_CSV, AUDIO_DEV)

del audio_model, feature_extractor
torch.cuda.empty_cache()
print("✅ Audio extraction complete! VRAM cleared.")

🚀 Đang tải Audio Model để trích xuất...


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

...

Processing audio for train_sent_emo.csv...


  0%|          | 0/9989 [00:00<?, ?it/s]2025-11-18 09:05:43.498690: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763456743.723721      39 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763456743.791776      39 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
100%|██████████| 9989/9989 [18:02<00:00,  9.22it/s]


Processing audio for dev_sent_emo.csv...


100%|██████████| 1109/1109 [01:59<00:00,  9.28it/s]

✅ Audio extraction complete! VRAM cleared.


In [5]:
# --- CELL 4: DATASET ---
class MultimodalDataset(Dataset):
    def __init__(self, csv_path, audio_features_dict):
        self.df = pd.read_csv(csv_path)
        self.audio_features = audio_features_dict
        self.tokenizer = AutoTokenizer.from_pretrained(TEXT_MODEL)
        
    def __len__(self): return len(self.df)
    
    def get_context_text(self, idx):
        row = self.df.iloc[idx]
        ctx = self.df[(self.df["Dialogue_ID"]==row["Dialogue_ID"]) & (self.df["Utterance_ID"]<row["Utterance_ID"])].tail(3)
        context = " </s> ".join([f"{r['Speaker']}: {r['Utterance']}" for _, r in ctx.iterrows()])
        return f"{context} </s> </s> {row['Speaker']}: {row['Utterance']}"

    def __getitem__(self, idx):
        text = self.get_context_text(idx)
        row = self.df.iloc[idx]
        uid = f"dia{row['Dialogue_ID']}_utt{row['Utterance_ID']}"
        
        enc = self.tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
        audio_vec = self.audio_features.get(uid, np.zeros(1024, dtype=np.float32))
        
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "audio_emb": torch.tensor(audio_vec, dtype=torch.float32),
            "label": torch.tensor(LABEL2ID[row["Emotion"]])
        }

In [8]:
# --- SỬA LẠI CELL 5 ---

class LoRAMultimodalModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        print("Loading RoBERTa-Large...")
        base_model = AutoModel.from_pretrained(TEXT_MODEL)
        
        # --- ⭐️ THAY ĐỔI QUAN TRỌNG: CẤU HÌNH MẠNH HƠN ⭐️ ---
        peft_config = LoraConfig(
            r=32,              # Tăng từ 16 lên 32 (Thông minh hơn)
            lora_alpha=64,     # Alpha thường = 2 * r
            lora_dropout=0.1,
            bias="none",
            task_type=TaskType.FEATURE_EXTRACTION,
            # Tấn công toàn diện: Train TẤT CẢ các lớp quan trọng của RoBERTa
            target_modules=["query", "key", "value", "dense"] 
        )
        # -----------------------------------------------------
        
        self.roberta = get_peft_model(base_model, peft_config)
        self.roberta.print_trainable_parameters() # Bạn sẽ thấy số tham số train tăng lên
        
        self.classifier = nn.Sequential(
            nn.Linear(1024 + 1024, 512), 
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, len(EMOTIONS))
        )
        
    def forward(self, input_ids, attention_mask, audio_emb):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        text_emb = outputs.last_hidden_state[:, 0, :] 
        combined = torch.cat((text_emb, audio_emb), dim=1)
        logits = self.classifier(combined)
        return logits

print("Kiến trúc LoRA (Target All Linear) đã sẵn sàng.")

Kiến trúc LoRA (Target All Linear) đã sẵn sàng.


In [9]:
# --- SỬA LẠI CELL 6 ---

BATCH_SIZE = 8
ACCUMULATION = 4
EPOCHS = 15 
LR = 1e-4   # <-- GIẢM TỪ 2e-4 XUỐNG 1e-4 ĐỂ ỔN ĐỊNH HƠN

# ... (Phần DataLoader giữ nguyên: num_workers=0) ...
train_ds = MultimodalDataset(TRAIN_CSV, audio_features_train)
val_ds = MultimodalDataset(DEV_CSV, audio_features_dev)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE*2, shuffle=False, num_workers=0)

# Init Model
model = LoRAMultimodalModel().to(device)
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=0.01)

# ... (Phần Scheduler và Loop giữ nguyên) ...
total_steps = len(train_loader) * EPOCHS // ACCUMULATION
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(0.1*total_steps), num_training_steps=total_steps)
loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1)

best_f1 = 0
print(f"🚀 Bắt đầu Fine-tuning RoBERTa (LoRA r=32, All-Linear) + Audio...")

for ep in range(EPOCHS):
    model.train()
    train_loss = 0
    
    for i, batch in enumerate(tqdm(train_loader, desc=f"Epoch {ep+1}")):
        input_ids = batch["input_ids"].to(device)
        mask = batch["attention_mask"].to(device)
        audio_emb = batch["audio_emb"].to(device)
        labels = batch["label"].to(device)
        
        logits = model(input_ids, mask, audio_emb)
        loss = loss_fn(logits, labels)
        
        loss = loss / ACCUMULATION
        loss.backward()
        
        if (i+1) % ACCUMULATION == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            
        train_loss += loss.item() * ACCUMULATION
        
    # Validate
    model.eval()
    all_preds, all_trues = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            mask = batch["attention_mask"].to(device)
            audio_emb = batch["audio_emb"].to(device)
            
            logits = model(input_ids, mask, audio_emb)
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            
            all_preds.extend(preds)
            all_trues.extend(batch["label"].numpy())
            
    val_f1 = f1_score(all_trues, all_preds, average="weighted")
    acc = accuracy_score(all_trues, all_preds)
    
    print(f"Epoch {ep+1}: Loss={train_loss/len(train_loader):.4f} | Val F1={val_f1:.4f} | Acc={acc:.4f}")
    
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), "best_lora_model.pt")
        print(f"✅ New Best F1: {best_f1:.4f} - Saved!")

print(f"🏆 KẾT QUẢ CUỐI CÙNG: {best_f1:.4f}")

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on

Loading RoBERTa-Large...
trainable params: 14,221,312 || all params: 369,581,056 || trainable%: 3.8480
🚀 Bắt đầu Fine-tuning RoBERTa (LoRA r=32, All-Linear) + Audio...


Epoch 1: 100%|██████████| 1249/1249 [11:01<00:00,  1.89it/s]


Epoch 1: Loss=1.6348 | Val F1=0.4870 | Acc=0.5528
✅ New Best F1: 0.4870 - Saved!


Epoch 2: 100%|██████████| 1249/1249 [11:01<00:00,  1.89it/s]


...

Epoch 5: Loss=0.9882 | Val F1=0.6483 | Acc=0.6438


Epoch 6: 100%|██████████| 1249/1249 [11:02<00:00,  1.88it/s]


Epoch 6: Loss=0.8982 | Val F1=0.6720 | Acc=0.6817


Epoch 7:   6%|▌         | 69/1249 [00:36<10:27,  1.88it/s]


KeyboardInterrupt: 